In [1]:
import os
os.chdir('../')

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import Lasso
from sklearn.externals import joblib
from urllib import urlencode
import lib.yelp_api as yp
import pandas as pd
import numpy as np
import requests
import json
import re

In [3]:
yelp_df = joblib.load('data/mtch_srch_rslts_{}.pkl'.format(0))

In [4]:
yelp_df.head(1)

,search_name,search_address,bus_id,name,price,cat_1,cat_2,closed,address,city,zip_code,latitude,longitude
brew-you-los-angeles,BREW YOU COFFEE,808 S WESTERN AVE,brew-you-los-angeles,Brew You,$,coffee,icecream,False,808 S Western Ave,Los Angeles,90005,34.05699,-118.30869


In [5]:
rest_df = joblib.load('data/la_rest_group_df.pkl')

In [6]:
rest_df.head(1)

,facility,address,city,zip_cd,num_at_address,max_seats,phone,risk_level,last_inspect_date,inspect_score,cat,closed_est,year
0,#1 CAFE,2080 CENTURY PARK E,LOS ANGELES,90067,1,30,+1nan,3,2016-08-18,90.0,None,False,2016


In [7]:
cmplt_rest_data = pd.merge(rest_df, yelp_df, left_on=['facility', 'address'], 
                           right_on=['search_name', 'search_address'], how='left')

cmplt_rest_data.head()

,facility,address_x,city_x,zip_cd,num_at_address,max_seats,phone,risk_level,last_inspect_date,inspect_score,...,name,price,cat_1,cat_2,closed,address_y,city_y,zip_code,latitude,longitude
0,#1 CAFE,2080 CENTURY PARK E,LOS ANGELES,90067,1,30,+1nan,3,2016-08-18,90.0,...,One Cafe,$$,cafes,cafes,False,2080 Century Park E,Los Angeles,90067,34.058765,-118.412033
1,#1 CHINESE FAST FOOD,8606 S VERMONT AVE,LOS ANGELES,90044,1,30,+12132358779,3,2016-08-10,92.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,#1 DONUT,8509 S FIGUEROA ST,LOS ANGELES,90003,2,30,+1nan,2,2016-05-17,98.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,#1 DONUTS,8509 S FIGUEROA ST,LOS ANGELES,90003,2,30,+16266957898,2,2016-07-26,91.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,#2 MOON BBQ,478 N WESTERN AVE,LOS ANGELES,90004,1,150,+13234656707,3,2016-10-12,92.0,...,Dal Dae Po,$,korean,korean,False,478 N Western Ave,Los Angeles,90004,34.079599,-118.308788


In [9]:
joblib.dump(cmplt_rest_data, 'data/tableau_data.csv')

['data/tableau_data.csv']

In [52]:
word_dict = {}
for rest in cmplt_rest_data['facility']:
    list_ = rest.split()
    for word in list_:
        if word in word_dict:
            word_dict[word] =+ 1
        else:
            word_dict[word] = 1

In [54]:
sorted(word_dict.items(), key=lambda x: x[1], reverse=True)

[('RESTAURANT', 707),
 ('CAFE', 568),
 ('THE', 424),
 ('&', 414),
 ('EL', 361),
 ('BAKERY', 324),
 ('LA', 320),
 ('PIZZA', 295),
 ('COFFEE', 288),
 ('BAR', 264),
 ('GRILL', 226),
 ('KITCHEN', 184),
 ('FOOD', 167),
 ('AND', 166),
 ('HOUSE', 160),
 ('EXPRESS', 160),
 ('SUBWAY', 151),
 ('DONUTS', 140),
 ('SUSHI', 132),
 ('BURGER', 131),
 ('THAI', 127),
 ('DELI', 120),
 ('CHICKEN', 112),
 ('KING', 105),
 ('TACO', 101),
 ('LOS', 98),
 ('STARBUCKS', 97),
 ('SHOP', 94),
 ('TACOS', 92),
 ('-', 92),
 ('MEXICAN', 91),
 ('MARKET', 90),
 ('BBQ', 81),
 ('TEA', 81),
 ('DONUT', 79),
 ('CHINESE', 79),
 ('JUICE', 79),
 ('LITTLE', 74),
 ('OF', 72),
 ('FISH', 69),
 ('CHINA', 68),
 ('BURGERS', 67),
 ('CUISINE', 64),
 ('IN', 63),
 ('MCDONALDS', 60),
 ('NEW', 59),
 ('CLUB', 58),
 ('FRIED', 58),
 ('LOUNGE', 57),
 ('BEAN', 57),
 ('ROOM', 56),
 ('LEAF', 56),
 ('BOX', 55),
 ('POLLO', 54),
 ('JACK', 49),
 ('DE', 47),
 ('LAS', 46),
 ('ON', 45),
 ('TAQUERIA', 45),
 ('A', 44),
 ('SEAFOOD', 43),
 ('HUT', 43),
 ('FRE

In [38]:
def compare_clsd_est(data, yelp):
    if yelp == 1 or yelp == 0:
        return 1
    else:
        return 0

print ('We estimated correctly on {}% of closings, \
given that we were able to actually find the \
correct restaurant through Yelp API.'.format(round(np.mean(cmplt_rest_data.apply(lambda col: compare_clsd_est(col['closed_est'],
                                                                              col['closed']), axis=1))),2))

We estimated correctly on 0.0% of closings, given that we were able to actually find the correct restaurant through Yelp API.


In [37]:
def compare_clsd_est(data, yelp):
    if yelp == 1 or yelp == 0:
        if data == yelp:
            return 100
        else:
            return 0
    else:
        pass

print ('We estimated correctly on {}% of closings, \
given that we were able to actually find the \
correct restaurant through Yelp API.'.format(round(np.mean(cmplt_rest_data.apply(lambda col: compare_clsd_est(col['closed_est'],
                                                                              col['closed']), axis=1))),2))

We estimated correctly on 81.0% of closings, given that we were able to actually find the correct restaurant through Yelp API.


## EDA

In [ ]:
x_ = zip_yr_clsd.index.values[:-1]
y_ = zip_yr_clsd['facility'].values[:-1]
hue_ = zip_yr_clsd['inspect_score'].values[:-1]

fig = plt.figure(figsize=(20,10))

sns.barplot(x=x_, y=y_, hue=hue_, palette='GnBu_d')
plt.xticks(rotation=45, ha='right', fontsize=14)
plt.xlabel('City', fontsize=20)
plt.yticks(fontsize=20)
plt.ylabel('Number of Facilities', fontsize=20)
plt.title('Scope of Data by City', fontsize=26)
plt.legend(bbox_to_anchor = (1, 1), loc = 2, handletextpad = 0.3, fontsize=14)
plt.show()

data = zip(y_, x_)
data.sort(reverse=True)

print data[0]
print data[1]
print data[2]
print data[3]
print data[4]

In [5]:
le = LabelEncoder()
yelp_df['price'] = le.fit_transform(yelp_df['price'].values)
yelp_df['cat_1'] = le.fit_transform(yelp_df['cat_1'].values)
yelp_df['cat_2'] = le.fit_transform(yelp_df['cat_2'].values)
yelp_df['closed'] = le.fit_transform(yelp_df['closed'].values)
yelp_df['city'] = le.fit_transform(yelp_df['city'].values)
yelp_df['zip_code'] = le.fit_transform(yelp_df['zip_code'].values)

In [6]:
yelp_df.head()

,search_name,search_address,bus_id,name,price,cat_1,cat_2,closed,address,city,zip_code,latitude,longitude
brew-you-los-angeles,BREW YOU COFFEE,808 S WESTERN AVE,brew-you-los-angeles,Brew You,1,32,75,0,808 S Western Ave,48,4,34.056990,-118.308690
boulevard-cafe-palmdale,BOULEVARD CAFE,2211 E PALMDALE BLVD,boulevard-cafe-palmdale,Boulevard Cafe,1,14,22,0,2211 E Palmdale Blvd,64,219,34.580845,-118.088768
bibi-sara-woodland-hills,BIBI,4878 TOPANGA CANYON BLVD,bibi-sara-woodland-hills,Bibi Sara,1,81,105,0,4878 Topanga Canyon Blvd,104,158,34.156490,-118.605675
bontobagi-glendale,BONTOBAGI,831 N PACIFIC AVE,bontobagi-glendale,Bontobagi,2,65,86,0,831 N Pacific Ave,30,131,34.158527,-118.264862
astro-karaoke-lomita,ASTRO,2150 LOMITA BLVD,astro-karaoke-lomita,Astro Karaoke,2,64,83,0,2150 Lomita Blvd,46,117,33.801670,-118.319008


In [7]:
X = yelp_df[['price', 'cat_1', 'cat_2', 'city',
             'zip_code', 'latitude', 'longitude']]
y = yelp_df['closed']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [9]:
l = Lasso()

In [10]:
l.fit(X_train, y_train)
l.score(X_train, y_train), l.score(X_test, y_test)

(0.00074761792689082007, -0.00028409493304426547)

In [11]:
dtc = DecisionTreeClassifier()

In [12]:
dtc.fit(X_train, y_train)
dtc.score(X_train, y_train), dtc.score(X_test, y_test)

(0.99606918238993714, 0.67294117647058826)